In [142]:
import math
import string
import operator as ops

tests = """
3:10**2
5:30
5+10
8*3
20/4
932//7
5+8.3
2**8
2^8
2^^3
"""

In [143]:
class Range:
    def __init__(self, start, stop=1, step=1):
        self.start = start
        self.stop = stop
        self.step = step
        self.value = None
        self.result_type = Array
        self.report = ['start', 'stop', 'step']
        
    def evaluate(self):
        self.value = list(range(self.start, self.stop+1, self.step))
        self.value = Array(self.value)
        return self.value

In [144]:
def tetration(n, m):
    r = n
    for i in range(m):
        r = n ** r
    return r

tetration(2, 3)

65536

In [145]:
class Expression:
    def __init__(self, a, b, op):
        self.a = a
        self.b = b
        self.op = op
        self.op_list = {
            '+': ops.add,
            '*': ops.mul,
            '-': ops.sub,
            '/': ops.truediv,
            '//': ops.floordiv,
            '**': ops.pow,
            '^': ops.pow,
            '^^': tetration,
        }
        self.report = ['a', 'b']
        
    def evaluate(self):
        op_name = self.op_list[self.op]
        self.value = op_name(self.a, self.b)
        return self.value

In [146]:
class Array:
    def __init__(self, terms):
        self.data = []
        for t in terms:
            if type(t) is list:
                self.data.append(array(t))
            else:
                if t not in list(',;'):
                    self.data.append(t)
                    
    def __str__(self):
        items = ', '.join(map(str, self.data))
        items = '['+items+']'
        return items

In [147]:
class Block:
    def __init__(self, components=None, parsed=None, parser=None, r=None, block_type=None):
        self.components = components
        self.parser = parser
        self.children = []
        self.dtype = None
        self.block_type = block_type
        self.type = block_type
        
        self.parsed = None
        if self.components:
            comp_vals = []
            for q in self.components:
                if type(q) in [Token]:
                    q = q.string
                    
                if type(q) in [Block]:
                    if q.like('num'):
                        q = int(q)
                elif q.isnumeric():
                    q = int(q)
                comp_vals.append(q)
            self.parsed = parser(*r(comp_vals))
            
    def add(self, x):
        self.children.append(x)
        
    def print(self, level=0):
        indent = ' '*2*level
#         print(self.type)
#         print(indent + str(self) + '; ' + type(self.parsed).__name__ + '; ' + str(self.type))
        print(indent + 'Block: ' + type(self.parsed).__name__ + '; ' + str(self.type))
        for c in self.children:
            c.print(level=level+1)
#         for k, v in vars(self.parser).items():
#             print('{}{}:{}'.format(indent, k, v))
        if self.parsed:
            for k in self.parsed.report:
                v = getattr(self.parsed, k)
                print('{}{}:{}'.format(indent, k, str(v)))
                
    def has_tokens(self):
        return any(type(b) is Token for b in self.children)
    
    def like(self, test):
        return self.type and (self.type == test or self.type.startswith(test))
        
    def __getitem__(self, i):
        return self.children[i]
    
    def __setitem__(self, i, j):
        self.children[i] = j

In [148]:
class Token(Block):
    def __init__(self, string='', token_type=None):
        super().__init__()
        self.string = string
        self.token_type = token_type
        self.type = self.token_type

In [149]:
class Program:
    def __init__(self, source):
        self.source = source
#         Create the root node
        self.tree = Block(block_type='root')
        
#         List of characters and their corresponding type
        self.char_sets = {
            'op': '!@#$%^&*/-+<>',
            'syntax': '()[]{},.;:=|',
            'letter': string.ascii_lowercase,
            'numeric': string.digits + '.-'
        }
#         List of syntactical patterns to match to generate the program's structure
#         i.e., the grammar
        self.patterns = {
#             'range': ['numeric', ':', 'numeric']
            'range': [
#                 lambda x: [x[0].like('num') and x[1] == ':' and x[2].like('num')],
                lambda x: x[0].like('num') and x[1] == ':' and x[2].like('num'),
                lambda x: [x[0], x[2]],
                Range
            ],
            'expression': [
                lambda x: x[1].like('op'),
                lambda x: [x[0], x[2], x[1]],
                Expression
            ]
        }
        
        self.lex()
        self.parse(self.tree)
        
    def char_type(self, x):
#         return list(filter(lambda x: k for k, v in self.char_sets if x in v))[0]
        return [k for k, v in self.char_sets.items() if x in v][0]
    
    def lex(self):
        statements = self.source.replace('|', '\n').split('\n')
#         Remove empty lines
        statements = list(filter(None, statements))
#         Loop through lines in program
        for s in statements:
            statement_parse = Block()
            token = Token(s[0], self.char_type(s[0]))
            for c in s[1:]:
                c_type = self.char_type(c)
#                 If character matches the type of the current token, append it
                if c_type == token.token_type:
                    token.string += c
#                 Otherwise, store the token and start a new one
                else:
                    statement_parse.add(token)
                    token = Token(c, c_type)
            statement_parse.add(token)
            self.tree.add(statement_parse)
    
    def parse(self, block, level=0):
        for j, b in enumerate(block.children):
            self.parse(b)
        i = 0
        for k, v in self.patterns.items():
            pattern, result, block_type = v
            num = 3
            section = block[i:i+num]
            if len(section) >= num:
#                     print(section, pattern(section))
                if pattern(section):
#                     print(True, block[i:i+num])
#                     block[i:i+num] = Block(section, parser=block_type, r=result, block_type='numeric')
                    block[i:i+num] = [Block(section, parser=block_type, r=result, block_type='numeric')]
#                 use has_tokens?
                    
    def execute(self, node=None):
#         Default node is the root node
        if not node:
            node = self.tree
        
#         Loop through nodes in tree
        for b in node:
            if b.parser in [Range, Expression]:
                b.parsed.evaluate()
                print(b.parsed.value)
#             Recursively execute subnodes
            else:
#                 for v in b:
#                     self.execute()
               self.execute(node=b)
                    
    def print_tree(self):
        self.tree.print()
    
        
M = Program(tests)
M.execute()
# vars(M.tree[0].parsed)
M.print_tree()
# M.tree[0].components[1].like('op')
# M.tree[-1].parser#.components[1].token_type

15
24
5.0
133
13
256
256
65536
Block: NoneType; root
  Block: NoneType; None
    Block: NoneType; numeric
    Block: NoneType; syntax
    Block: NoneType; numeric
    Block: NoneType; op
    Block: NoneType; numeric
  Block: NoneType; None
    Block: NoneType; numeric
    Block: NoneType; syntax
    Block: NoneType; numeric
  Block: NoneType; None
    Block: Expression; numeric
    a:5
    b:10
  Block: NoneType; None
    Block: Expression; numeric
    a:8
    b:3
  Block: NoneType; None
    Block: Expression; numeric
    a:20
    b:4
  Block: NoneType; None
    Block: Expression; numeric
    a:932
    b:7
  Block: NoneType; None
    Block: Expression; numeric
    a:5
    b:8
    Block: NoneType; syntax
    Block: NoneType; numeric
  Block: NoneType; None
    Block: Expression; numeric
    a:2
    b:8
  Block: NoneType; None
    Block: Expression; numeric
    a:2
    b:8
  Block: NoneType; None
    Block: Expression; numeric
    a:2
    b:3


In [97]:
# M.__dict__
M.tree[0][1].type

'op'

In [68]:
bool([False])
f = [3, 7, 2, 1, 8]
f[0:3]

[3, 7, 2]